In [1]:
import sys
import os

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

블로그 리뷰 url 크롤링

In [ ]:
# 여러 동에 대한 반복문
dong_list = ['연희동', '후암동', '망원동', '연남동', '성수동']
for dong in dong_list:
    # 각 동에 해당하는 카페 리스트 파일 읽기
    cafe_list_path = f'/Users/hana/MJU/3-2/빅데이터프로그래밍/BDP_Proj/cafelist/cafelist_{dong}.csv'
    cafe_list = pd.read_csv(cafe_list_path)
    cafe_list = cafe_list.drop("Unnamed: 0", axis=1)
    cafe_list = cafe_list.drop("cafe_type", axis=1)
    
    for cafe in cafe_list["name"]:
        dong_name = dong.replace('동', '')
        if dong_name + '점' in cafe:
            cafe = cafe.replace(dong_name + '점', '')
        
        cafe = dong_name + " " + cafe
        cafe_keywords = cafe.split()
    
        # 파일 경로 설정
        folder_path = f"BDP-Project/{dong}/"
        file_path = os.path.join(folder_path, f"{cafe}_블로그_리뷰.csv")
        if not os.path.exists(file_path):

            #크롬 웹브라우저 실행
            chrome_options = webdriver.ChromeOptions()
            
            chrome_options.add_argument("/Users/hana/.wdm/drivers/chromedriver/mac64/119.0.6045.105/chromedriver-mac-arm64/chromedriver")
            driver = webdriver.Chrome()
            driver.get(
                "https://search.naver.com"
            )
            time.sleep(1)
            
            cafe_keywords = cafe.split()
    
            #검색창에 "검색어" 검색
            element = driver.find_element(By.CSS_SELECTOR, 'div.search_input_box > input#query')
            element.send_keys(dong+" "+cafe)
            element.submit()
            time.sleep(1)
        
            #VIEW 클릭
            driver.find_element(By.LINK_TEXT, "VIEW").click()
            
            #옵션 클릭
            driver.find_element(By.LINK_TEXT, "옵션").click()
            
            #블로그 클릭
            driver.find_element(By.LINK_TEXT, "블로그").click()
        
            url_list = []
            title_list = []
        
            # 스크롤을 밑으로 내려주는 함수
            def scroll_down(driver):
                driver.execute_script("window.scrollTo(0, 99999999)")
                time.sleep(1)
            
                # n : 스크롤할 횟수 설정
                n = 5 #스크롤 1번당 글 30개씩 화면에 출력
                i = 0
                while i < n:
                    scroll_down(driver) #스크롤다운
                    i = i+1
            
            
            #URL_raw 크롤링
            articles = "a.title_link"
            article_raw = driver.find_elements(By.CSS_SELECTOR, articles)
            for article in article_raw:
                title = article.text
        
                if all(keyword in title for keyword in cafe_keywords):
                    url = article.get_attribute('href')
                    url_list.append(url)
                    title_list.append(title)
        
                else:
                    continue
                    
            # 10개 미만으로 게시글을 가질 경우 검색 결과 상위 10개 추출
            if (len(url_list) < 10):
                url_list = []
                title_list = []

                article_raw = article_raw[:10]
                
                #URL_raw 크롤링
                for article in article_raw:
                    url = article.get_attribute('href')
                    url_list.append(url)
                
                    title = article.text
                    title_list.append(title)

            print("")
            print(cafe)
            print('url 갯수: ',len(url_list))
            print('title 갯수: ',len(title_list))
            
            df = {}
            df = pd.DataFrame({'url':url_list, 'title':title_list})
            
            # 폴더가 존재하지 않으면 생성
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # 파일명 지정 (i 변수를 사용하여 동적으로 파일명 생성)
            file_name = f"{cafe}_블로그_리뷰.csv"
            
            # 파일 전체 경로 생성
            file_path = os.path.join(folder_path, file_name)
            
            # 데이터프레임을 CSV 파일로 저장
            df.to_csv(file_path, index=False)
            driver.close()
            
        else:
            print(f"File {file_path} already exists. Skipping…\n")
    
print(f"Crawling for {dong} completed.\n")

블로그 리뷰 콘텐츠 크롤링

In [3]:

# ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("/Users/hana/.wdm/drivers/chromedriver/mac64/119.0.6045.105/chromedriver-mac-arm64/chromedriver")

# 크롬 웹브라우저 실행
driver = webdriver.Chrome()
driver.get("https://search.naver.com")
time.sleep(1)

dong_list = ['망원동']

for dong in dong_list:
    # 폴더 경로 설정
    folder_path = f"/Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_url_list/{dong}/"
    
    # 폴더 내 파일 목록 가져오기
    file_list = os.listdir(folder_path)
    
    # 파일 순차적으로 읽기
    for file_name in file_list:
        cafe_name = file_name.split('_')[0]
        
        
        # 결과 저장 파일 경로 읽기 
        result_folder_path = f"/Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/{dong}/"
        in_result_file_path = os.path.join(result_folder_path, f"{cafe_name}_blog_content.csv")
        
        if not os.path.exists(in_result_file_path):
            print(file_name)
        
            # 파일 읽기 예시 (pandas를 사용하여 CSV 파일을 읽는 경우)
            if file_name.endswith(".csv"):
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path)
                
                result_dict = {}
    
                number = len(df)
                
                for i in tqdm_notebook(range(number)):
                    # 글 띄우기
                    url = df['url'][i]
                    driver.get(url)
                    
                    try:
                        driver.switch_to.frame('mainFrame')
                
                        target_info = {}
                
                        # 제목 크롤링
                        overlays = ".se-module.se-module-text.se-title-text"
                        tit = driver.find_element(By.CSS_SELECTOR, overlays)
                        title = tit.text
                
                        # 내용 크롤링
                        overlays = ".se-main-container"
                        content = driver.find_element(By.CSS_SELECTOR, overlays)
                        cont = content.text
                        cont = cont.replace('\n', ' ')
                
                        # 딕셔너리에 담음
                        target_info['title'] = title
                        target_info['content'] = cont
                
                        # 전체 딕셔너리에 담음
                        result_dict[i] = target_info
                        time.sleep(1)
                
                        print(i, title)

                    except:
                        continue
                    
                print('수집글 갯수', len(result_dict))
                result = pd.DataFrame.from_dict(result_dict, orient='index')

                # 결과 저장 폴더가 존재하지 않으면 생성
                if not os.path.exists(result_folder_path):
                    os.makedirs(result_folder_path)

                # 파일명 지정 (i 변수를 사용하여 동적으로 파일명 생성)
                result_file_name = f"{cafe_name}_blog_content.csv"
                
                # 파일 전체 경로 생성
                result_file_path = os.path.join(result_folder_path, result_file_name)
                
                # 데이터프레임을 CSV 파일로 저장
                result.to_csv(result_file_path, index=False)
                print(" ")
                time.sleep(3)

        else:
            print(f"File {in_result_file_path} already exists. Skipping…\n")

# 마무리 작업
driver.close()
print("All files processed.")


File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 604seoul_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 79파운야드 망원본점_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 9ram_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 AIT_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 Alive214_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 BGT호두단팥빵 망원동점_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_lis

  0%|          | 0/171 [00:00<?, ?it/s]

0 [망원동카페] 퍼스널슈페너와 버터바가 맛있는 퍼스널커피
1 망원동카페 <퍼스널커피> 인생라떼 찾았음!
2 [망원]내가 좋아하는 망원동 분위기_퍼스널커피/망리단길상권
3 버터바가 맛있었던 조용한 망원동카페 ‘퍼스널커피’(영업시간/유자 얼그레이/ 라떼맛집)
4 망리단길 망원동 카페 퍼스널커피 제대로 된 디저트 강자
5 망원동카페 :: 스콘이 맛있는 '퍼스널커피'
6 망원동카페 퍼스널커피 PERSONAL COFFEE
7 [망원역 카페] 퍼스널커피⎮망원동약과쿠키 맛집, 망원동버터바 맛집 (위치, 메뉴판)
8 [망원동 카페] 망리단길 스콘 맛집 조용한 카페 추천 :: 퍼스널커피
9 망원동 카페, 퍼스널커피 인생 버터바 (반려동물 동반)
10 크림치즈 약과쿠키 망원동 퍼스널커피
11 [퍼스널커피] 망원동 카페 // 디카페인 맛집
12 망원동 카페 쿠키맛집 퍼스널커피 n번째방문 찐후기
13 [망원동카페: 망원역] 망원동버터바 망원동약과쿠키, 퍼스널커피
14 [망원동 카페] 퍼스널커피/ 퍼스널 슈페너 맛집이네... 꾸덕한 크림이 존맛탱구리
15 망원동 혼자가기 좋은 카페 퍼스널커피(수제 약과쿠키, 버터바 맛집)
16 망리단길 퍼스널커피 망원동카페 민트초코 버터바
17 망원동 꾸덕한 버터바 와 고소한 크림 라떼 맛집 [ 퍼스널커피 ] 마포구 로스터리 카페
18 망원동 약과쿠키와 버터바 맛집 퍼스널커피
19 망원동카페 퍼스널커피 (Personal Coffee)
20 조용하고 깔끔해서 혼커하기 좋은 망원동 애견동반카페 퍼스널커피
21 퍼스널커피 예쁘고 맛있는 여유로운 망원동카페
22 서울 망원동 퍼스널커피 스콘까지 맛있는 망원동 카페
23 [마포,망원] 망원동카페 민초단이라면 지나칠 수 없는 곳. 민트초코 버터바가 있는 ‘퍼스널커피’
24 망원동 약과 쿠키 맛집! 꾸덕한 버터바가 있는 로스터리 카페, 퍼스널커피.
25 망원동카페) 퍼스널커피 PERSONAL COFFEE (feat. 퍼스널슈페너, 솔티 카라멜 버터바 맛집 / 로스터리 카페)
26 [망원] 커피&디저트 맛있는 망

  0%|          | 0/90 [00:00<?, ?it/s]

0 (망원동) 페스토 페스토
1 데이트하기 좋은 망원동 페스토 페스토에서 바질피자 먹었어요.
2 [망원동] 페스토 페스토
3 [망원동 맛집] 페스토 국내완탑👍🏻 <페스토 페스토>
4 망원동 맛집 페스토페스토에서 계절 음식 맛보기
5 망원동 페스토페스토 - 수제 페스토 향기가 가득한 기분 좋은 가게
6 [망원동 페스토 페스토] 바질 페스토 이태리 요리 맛집
7 [서울/망원동] 힙한 분위기의 샌드위치집 | 피자집 '페스토페스토'
8 망원동 맛집: 페스토 페스토 -건강한 페스토와 샌드위치, 바질 소면
9 망원동 숨은 신상맛집 , 페스토페스토 PEPE 네추럴와인 주말데이트 feat. 이태리반찬가게
10 망원동 맛집__페스토페스토(Pesto Pesto)
11 [망원동 맛집] 페스토 페스토 | 망원 페스토 음식 , 페스토 맛집
12 망원동 놀거리 데이트 와인바 망원 페스토페스토 분위기 최고!
13 [망원동 맛집]페스토 페스토 - 바질페스토로 끝장보는 페스트리피자 맛집
14 [망원동 맛집/카페/비건/ 데이트] 페스토페스토, 고미푸딩
15 [망원동] 만두란, 페스토 페스토
16 망원동 맛집 ｜ 페스토페스토
17 망원동 프레시한 바질맛집 <페스토 페스토> (메뉴,주차,Tip)
18 망원동 맛집 페스토 페스토, 맛보지 않으면 후회할 이색 요리들
19 분위기 좋은 망원동 이탈리안 식당 페스토페스토
20 망원동 비건 브런치 맛집 / 페스토 페스토
21 망원동 주민픽 맛집 페스토페스토 (Pesto Pesto)에서 피자 먹은 후기!
22 [망원동 비건 맛집] 페스토 페스토
23 망원동 향긋한 허브 이탈리아 요리 페스토 페스토
24 페스토 페스토, 망원동 : 다양한 페스토를 활용한 음식을 맛볼수 있는 곳
25 망원동 페스토 전문 비건 음식 존맛,, - 페스토 페스토
26 페스토를 활용한 다양한 요리를 맛볼 수 있는 망원동 찐맛집, 페스토페스토 (pesto pesto)
27 망원동 맛집 페스토 페스토, 페스토 포장 추석선물 추천
28 망원동 페스토페스토 :: 친환경 페스토 전문점
29 망

...

0 [서울] 망원동 후와후와 소금빵, 샌드베이글
1 망원동 일본빵 / 베이글 전문 베이커리🍘 후와후와
2 망원동 빵집 소금빵 맛집 “후와후와”
3 망원동 빵집 후와후와 소금빵 베이글 샌드
4 망원동 빵집 후와후와 일본풍 분위기의 겉바속촉 소금빵 맛집
5 망원동 베이커리/소금빵 맛집/후와후와
6 망원동 후와후와
7 망원동 빵집 [후와후와] : 베이글 맛집, 소금빵 맛집
8 베이글이 맛있는 일본 감성의 귀여운 망원동 신상 빵집_ 후와후와
9 망원동 신상 빵집 후와후와
10 망원동 빵집 소금빵 베이글 맛집 후와후와 다녀옴
11 망원동 신상 소금빵집 후와후와
12 [서울] 망원동 신상빵집 후와후와 소금빵, 샌드베이글 후기
13 망원동 신상 빵집 ＂후와후와＂:: 베이글 도른놈, 크림치즈 무서운놈, 식빵 미친놈, 소금빵 기다려 또 간다.
수집글 갯수 14
 
망원동 흐쎄트_블로그_리뷰.csv


  0%|          | 0/20 [00:00<?, ?it/s]

0 망원동 흐쎄트(테두리없는 우유식빵이시그니쳔데 까눌레랑 스콘이 재방문각)
1 망원동빵집 추천 :: 흐쎄트
2 망원동빵집 : 빵지순례로 꼭 가야할 디저트 맛집 흐쎄트
3 망리단길 빵집, 디저트맛집 망원동빵집 흐쎄트
4 맛있는 빵이 한가득 망원동빵집 흐쎄트
5 최애 망원동빵집 '흐쎄트' 그냥 지나칠 수 없지~!
6 반해버린 망원동빵집 / 흐쎄트
7 망원역 근처 디저트맛집 망원동빵집 [흐쎄트]
8 #12 망원동 '흐쎄트'
9 망원동빵집 흐쎄트 / 새로생긴 신상 망원동디저트 맛집 !
10 망원동빵맛집 망원동빵집 [흐쎄트]
11 빵순이 빵돌이 망원동빵집 흐쎄트로 모여라~
12 빵순이가 좋아하는 망원동빵집 [흐쎄트]
13 망리단길 디저트 맛집 [ 망원동빵집 :: 흐쎄트 ]
14 망원동빵집 망리단길빵집 망리단길디저트 흐쎄트
15 테이크아웃 빵집 구움과자 맛집 망원동빵집 흐쎄트
16 망원동빵집 :: 빵순이 필수코스 베이커리 맛집 흐쎄트
17 맛있고 종류가 다양한 망원동빵집 / 흐쎄트
18 까눌레 맛집 ::::: 망원동빵집 / 흐쎄트
19 [망원동빵집/흐쎄트] 테이크아웃 전문♡ 우유식빵 유명한곳 다녀온 후기!
수집글 갯수 20
 
망원동 히카_블로그_리뷰.csv


  0%|          | 0/31 [00:00<?, ?it/s]

0 망원동 망리단길 카페 히카 - 커피와 케이크가 맛있는 감성 카페
1 망원 히카 : 산뜻한 크림이 매력적인 망원동 케이크 맛집
2 [망원동카페] 망원동으로 이전한 ‘히카카페’ 드디어 방문!
3 망원동 디저트 카페 히카 케이크 구매후기
4 인천 롯데백화점 팝업 망원동 히카 케이크
5 망원동 수제케익이 맛있는 히카 카페
6 망원동 수제 케이크 하면 찾게 된다는 디저트 맛집 카페 히카
7 [망원] 다양한 수제케이크를 파는 망원동 히카
8 [서울 망원동] 히카(HICA) | 수제케이크 맛집 애견동반카페 | 비건 옵션 제공
9 망원동 카페 히카
10 망원동 카페 애견동반 가능한 케이크가 맛있는 카페히카 HICA
11 [일상기록] 8월의 빵순이 일상 - 부천애즈브로우/휘낭시에 맛집 카페리븐/망원동 케이크 맛집 히카 팝업/백운신상카페 너티크림
12 [망원동히카] 생크림케잌이 맛있는 망원동 히카(Hica)/강아지동반카페☕️🐶
13 [망원동] 히카 케이크🎂 - 얼그레이 자몽 HICA CAKE (당일 예약, 할인 받는 방법)
14 [방문]홍대 빤쮸토끼 팝업 | 또보겠지떡볶이 | 망원동 히카 | 용산 도라애몽 팝업 카페
15 [망원동 카페] 케이크가 맛있는 카페 히카 Hica
16 [망원동/카페] 히카 HICA (신상맛집/반려동물 동반 가능/내돈내산/수제케이크)
17 망원동 케이크 맛집 카페 히카
18 상업용) 서울시 마포구 망원동 카페 / 커피숍 / 히카 / 케이크전문점 바닥시공 (장판시공/데코타일시공/마루시공)
19 [서울/ 망원동] 카페 히카!
20 카페 히카 망원동 애견동반 가능한디저트 카페
21 망원동 카페 추천 : 수제케이크맛집 히카
22 망원동 애견동반카페 '히카'에서 인생케이크
23 서울 망원동 카페_히카(HICA)에서 얼그레이자몽케익 사기
24 [망원동] 당충전! 행복충전! 망원동 케이크맛집🍰 카페 '히카'_인천롯백 팝업
25 [망원동 카페/ 케이크 맛집] 진짜 너무 맛있다. 히카
26 망원동 카페 케이크 맛집 히카 HICA
27 [망원동 카페] 망리단길 케이크